In [1]:
from openpyxl import load_workbook
from tqdm import tqdm
import csv

# 讀取 Excel 檔案
file_path = "113-1 NTU Course.xlsx"
wb = load_workbook(file_path)
ws = wb.active  # 或 wb["你的工作表名稱"]

In [3]:
# 準備儲存所有資料（包含 E 欄加入的超連結）
data = []

# 遍歷列數
for row in tqdm(range(1, ws.max_row + 1), desc="處理中"):
    row_data = []
    for col in range(1, 6):  # 讀取 A~E 欄（1~5）
        row_data.append(ws.cell(row=row, column=col).value)

    # 將超連結（若有）寫入第 5 欄（E欄）
    cell = ws.cell(row=row, column=2)  # B欄是 column 2
    if cell.hyperlink:
        link = cell.hyperlink.target
        row_data[4] = link  # 將第 5 欄覆蓋為超連結
    data.append(row_data)

# 將資料儲存為 CSV 檔
csv_file = "113-1_NTU_Course_with_links.csv"
with open(csv_file, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerows(data)

print(f"✅ 已儲存為：{csv_file}")

處理中: 100%|██████████| 1301/1301 [00:00<00:00, 57759.70it/s]

✅ 已儲存為：113-1_NTU_Course_with_links.csv


In [ ]:
import threading
import requests
import queue
import csv
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm.notebook import tqdm

# ===== 設定區 =====
URL_CSV = '113-1_NTU_Course_with_links.csv'
OUTPUT_CSV = 'course_data.csv'
ERROR_LOG = 'error_urls.csv'
MAX_FETCH_THREADS = 5
MAX_PARSE_THREADS = 5
HEADERS = {'User-Agent': 'Mozilla/5.0'}

# ===== 初始化資料與 queue =====
df = pd.read_csv(URL_CSV)
fetch_queue = queue.Queue()
parse_queue = queue.Queue()
writer_queue = queue.Queue()
error_list = []

for url in df["課程連結"]:
    fetch_queue.put(url)

# ===== tqdm 記錄數量 =====
progress_lock = threading.Lock()
fetch_pbar = tqdm(total=fetch_queue.qsize(), desc="Fetched", position=0)
parse_pbar = tqdm(total=fetch_queue.qsize(), desc="Parsed", position=1)
write_pbar = tqdm(total=fetch_queue.qsize(), desc="Written", position=2)

# ===== 抓取階段 =====
def fetcher():
    while True:
        try:
            url = fetch_queue.get(timeout=3)
        except queue.Empty:
            break
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            parse_queue.put((url, response.text))
        except Exception as e:
            print(f"[抓取失敗] {url}: {e}")
            error_list.append((url, 'fetch', str(e)))
        finally:
            with progress_lock:
                fetch_pbar.update(1)
            fetch_queue.task_done()

# ===== 解析階段 =====
def parser():
    while True:
        try:
            url, html = parse_queue.get(timeout=30)
        except queue.Empty:
            print("Parse queue is empty.")
            break
        try:
            soup = BeautifulSoup(html, 'html.parser')
            # article = soup.find(id='article_page')
            text = soup.get_text(separator="\n", strip=True) if soup else ""
            writer_queue.put((url, text))
        except Exception as e:
            print(f"[解析失敗] {url}: {e}")
            error_list.append((url, 'parse', str(e)))
        finally:
            with progress_lock:
                parse_pbar.update(1)
            parse_queue.task_done()

# ===== 寫入階段 =====
def writer():
    with open(OUTPUT_CSV, 'w', newline='', encoding='utf-8') as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(["課程連結", "text"])
        while True:
            try:
                url, text = writer_queue.get(timeout=30)
                csv_writer.writerow([url, text])
                with progress_lock:
                    write_pbar.update(1)
                writer_queue.task_done()
            except queue.Empty:
                print("Writer queue is empty.")
                break

# ===== 錯誤寫入 =====
def write_errors():
    if error_list:
        with open(ERROR_LOG, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['url', 'stage', 'error'])
            writer.writerows(error_list)

# ===== 執行流程控制 =====
if __name__ == '__main__':
    start_time = time.time()

    # 啟動所有 thread（平行）
    fetch_threads = [threading.Thread(target=fetcher) for _ in range(MAX_FETCH_THREADS)]
    parse_threads = [threading.Thread(target=parser) for _ in range(MAX_PARSE_THREADS)]
    writer_thread = threading.Thread(target=writer)

    for t in fetch_threads + parse_threads:
        t.start()
    writer_thread.start()

    for t in fetch_threads:
        t.join()
    fetch_queue.join()

    for t in parse_threads:
        t.join()
    parse_queue.join()

    writer_thread.join()
    writer_queue.join()

    # 寫入錯誤紀錄
    write_errors()

    fetch_pbar.close()
    parse_pbar.close()
    write_pbar.close()

    end_time = time.time()
    print(f"✅ 全部完成，用時 {end_time - start_time:.2f} 秒")

In [5]:
import threading
import requests
import queue
import csv
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm.notebook import tqdm

# ===== 設定區 =====
URL_CSV = "113-1_NTU_Course_with_links.csv"
OUTPUT_CSV = "course_data.csv"
ERROR_LOG = "error_urls.csv"
MAX_FETCH_THREADS = 5
MAX_PARSE_THREADS = 5
HEADERS = {"User-Agent": "Mozilla/5.0"}

# ===== 初始化資料與 queue =====
df = pd.read_csv(URL_CSV)
fetch_queue = queue.Queue()
parse_queue = queue.Queue()
writer_queue = queue.Queue()
error_list = []

for _, row in df.iterrows():
    fetch_queue.put(row.to_dict())

# ===== tqdm 記錄數量 =====
progress_lock = threading.Lock()
fetch_pbar = tqdm(total=fetch_queue.qsize(), desc="Fetched", position=0)
parse_pbar = tqdm(total=fetch_queue.qsize(), desc="Parsed", position=1)
write_pbar = tqdm(total=fetch_queue.qsize(), desc="Written", position=2)


# ===== 抓取階段 =====
def fetcher():
    while True:
        try:
            row = fetch_queue.get(timeout=3)
            url = row["課程連結"]
        except queue.Empty:
            break
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            parse_queue.put((row, response.text))
        except Exception as e:
            print(f"[抓取失敗] {url}: {e}")
            error_list.append((url, "fetch", str(e)))
        finally:
            with progress_lock:
                fetch_pbar.update(1)
            fetch_queue.task_done()


# ===== 解析階段 =====
def parser():
    while True:
        try:
            row, html = parse_queue.get(timeout=30)
            url = row["課程連結"]
        except queue.Empty:
            print("Parse queue is empty.")
            break
        try:
            soup = BeautifulSoup(html, "html.parser")
            text = soup.get_text(separator="\n", strip=True) if soup else ""
            writer_queue.put((row, text))
        except Exception as e:
            print(f"[解析失敗] {url}: {e}")
            error_list.append((url, "parse", str(e)))
        finally:
            with progress_lock:
                parse_pbar.update(1)
            parse_queue.task_done()


# ===== 寫入階段 =====
def writer():
    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8-sig") as f:
        csv_writer = None
        while True:
            try:
                row, text = writer_queue.get(timeout=30)
                row["text"] = text
                if csv_writer is None:
                    # 初始化欄位名稱
                    fieldnames = list(row.keys())
                    csv_writer = csv.DictWriter(f, fieldnames=fieldnames)
                    csv_writer.writeheader()
                csv_writer.writerow(row)
                with progress_lock:
                    write_pbar.update(1)
                writer_queue.task_done()
            except queue.Empty:
                print("Writer queue is empty.")
                break


# ===== 錯誤寫入 =====
def write_errors():
    if error_list:
        with open(ERROR_LOG, "w", newline="", encoding="utf-8-sig") as f:
            writer = csv.writer(f)
            writer.writerow(["url", "stage", "error"])
            writer.writerows(error_list)


# ===== 執行流程控制 =====
if __name__ == "__main__":
    start_time = time.time()

    fetch_threads = [threading.Thread(target=fetcher) for _ in range(MAX_FETCH_THREADS)]
    parse_threads = [threading.Thread(target=parser) for _ in range(MAX_PARSE_THREADS)]
    writer_thread = threading.Thread(target=writer)

    for t in fetch_threads + parse_threads:
        t.start()
    writer_thread.start()

    for t in fetch_threads:
        t.join()
    fetch_queue.join()

    for t in parse_threads:
        t.join()
    parse_queue.join()

    writer_thread.join()
    writer_queue.join()

    write_errors()

    fetch_pbar.close()
    parse_pbar.close()
    write_pbar.close()

    end_time = time.time()
    print(f"✅ 全部完成，用時 {end_time - start_time:.2f} 秒")

Fetched:   0%|          | 0/1300 [00:00<?, ?it/s]

Parsed:   0%|          | 0/1300 [00:00<?, ?it/s]

Written:   0%|          | 0/1300 [00:00<?, ?it/s]

Parse queue is empty.Parse queue is empty.

Parse queue is empty.
Parse queue is empty.
Writer queue is empty.
Parse queue is empty.
✅ 全部完成，用時 1090.85 秒
